In [9]:
import pandas as pd
import random
import psycopg2
from psycopg2.extras import execute_values

In [10]:
nomes = {
    "Ana", "Bruno", "Carla", "Daniel", "Eduarda", "Fernando", "Gabriela", "Hugo", "Isabela", "João",
    "Karina", "Lucas", "Mariana", "Nathan", "Olivia", "Paulo", "Quezia", "Rafael", "Sofia", "Tiago",
    "Ursula", "Vitor", "Wellington", "Ximena", "Yago", "Zara", "Alice", "Benjamin", "Camila", "Diego",
    "Elisa", "Fábio", "Giovana", "Heitor", "Ingrid", "Júlio", "Katia", "Leandro", "Mirela", "Norberto",
    "Orlando", "Priscila", "Quirino", "Raquel", "Samuel", "Tainá", "Ubirajara", "Valéria", "William", "Xavier",
    "Yasmin", "Zaqueu", "Adriana", "Bernardo", "Cecília", "Douglas", "Esther", "Felipe", "Gisele", "Henrique",
    "Ivana", "Jonas", "Kelly", "Leonardo", "Melissa", "Nicolas", "Otávio", "Patrícia", "Quintino", "Rebeca",
    "Simone", "Tatiane", "Ulisses", "Vanessa", "Wesley", "Xandra", "Yuri", "Zélia", "André", "Bianca", "Cristiano",
    "Diana", "Enzo", "Fernanda", "Gustavo", "Helena", "Ícaro", "Jéssica", "Kleber", "Lorena", "Matheus", "Natália",
    "Orfeu", "Paloma", "Quésia", "Rodrigo", "Selma", "Tarcísio", "Uriel", "Vitória", "Wallace", "Xênia", "Yvone",
    "Zilda", "Augusto", "Bárbara", "Cauê", "Débora", "Emanuel", "Flávia", "Gilberto", "Hilda", "Ísis", "Jefferson",
    "Karen", "Luana", "Maurício", "Neide", "Osvaldo", "Pedro", "Quitéria", "Ricardo", "Samantha", "Tamara", "Ugo",
    "Vilma", "Wanderley", "Xisto", "Ylana", "Zoroastro", "Aline", "Brenda", "Caio", "Denise", "Elton", "Fabiana",
    "Geraldo", "Heloísa", "Ítalo", "Joana", "Kevin", "Lívia", "Marcelo", "Nair", "Olavo", "Paula", "Quirina", "Roberto",
    "Sandro", "Tereza", "Urbano", "Vânia", "Wilson", "Xande", "Yasmina", "Zenaide"
}

sobrenomes = {
    "Silva", "Santos", "Oliveira", "Souza", "Pereira", "Lima", "Carvalho", "Ferreira", "Rodrigues", "Almeida",
    "Costa", "Gomes", "Martins", "Barros", "Vieira", "Mendes", "Nunes", "Moreira", "Teixeira", "Cavalcante",
    "Monteiro", "Dias", "Correia", "Azevedo", "Ribeiro", "Cardoso", "Freitas", "Duarte", "Borges", "Araújo",
    "Andrade", "Moura", "Xavier", "Figueiredo", "Cunha", "Torres", "Rezende", "Siqueira", "Batista", "Neves"
}

In [11]:
def nome_aleatorio():
    return f"{random.choice(tuple(nomes))} {random.choice(tuple(sobrenomes))}"

In [12]:
PATH = './parquets'

In [13]:
CHUNK_SIZE = 1000

In [14]:
INSERT_SQL = """INSERT INTO users (id, nome, embedding) VALUES %s ON CONFLICT (id) DO NOTHING"""

In [15]:
DB_PARAMS = {
    "dbname": "db",
    "user": "user",
    "password": "pass",
    "host": "localhost",
    "port": 5416
}

In [16]:
users = pd.read_parquet(f'{PATH}/df_users.parquet')

In [17]:
users.rename(columns={'userId':'id', 'user_weighted_embedding':'embedding'}, inplace=True)

In [18]:
users['nome'] = users.index.map(lambda _: nome_aleatorio())

In [19]:
users.head()

,id,embedding,nome
0,000044b36375e7f1a66a9476affc2ddc83c2ec6dd18951...,"[0.35516560769347333, 0.14411123629151734, -0....",Elton Dias
1,00007a4e5949a3dba7c977503c53e0873643fe17d0802a...,"[0.5453204643662357, -0.2766114007783694, -0.1...",Henrique Andrade
2,00011b1ced626112372206634e0e9b5ccb432da916e83f...,"[0.2522570172336757, -0.1293059356119698, -0.0...",Lívia Santos
3,00012cd490cea9fe043146807def18c830fa0d0b791772...,"[0.29292065552608554, 0.0649563024426402, 0.12...",Yago Santos
4,0001b40676c18a37bf25f0b1921ad12513c434cb57db18...,"[0.23972679953425952, -0.13138856623615228, -0...",Simone Costa


In [20]:
users = users[['id', 'nome', 'embedding']]

In [21]:
users.head()

,id,nome,embedding
0,000044b36375e7f1a66a9476affc2ddc83c2ec6dd18951...,Elton Dias,"[0.35516560769347333, 0.14411123629151734, -0...."
1,00007a4e5949a3dba7c977503c53e0873643fe17d0802a...,Henrique Andrade,"[0.5453204643662357, -0.2766114007783694, -0.1..."
2,00011b1ced626112372206634e0e9b5ccb432da916e83f...,Lívia Santos,"[0.2522570172336757, -0.1293059356119698, -0.0..."
3,00012cd490cea9fe043146807def18c830fa0d0b791772...,Yago Santos,"[0.29292065552608554, 0.0649563024426402, 0.12..."
4,0001b40676c18a37bf25f0b1921ad12513c434cb57db18...,Simone Costa,"[0.23972679953425952, -0.13138856623615228, -0..."


In [22]:
try:
    conn = psycopg2.connect(**DB_PARAMS)
    cur = conn.cursor()

    for i in range(0, len(users), CHUNK_SIZE):
        chunk = users.iloc[i:i + CHUNK_SIZE]
        values = [(x[0],x[1],x[2].tolist()) for x in chunk.to_numpy()]
        execute_values(cur, INSERT_SQL, values)
        conn.commit()
        print(f"Chunk {i // CHUNK_SIZE + 1} inserido com sucesso.")

    cur.close()
    conn.close()
    print("Inserção concluída!")

except Exception as e:
    print(f"Erro: {e}")

Chunk 1 inserido com sucesso.
Chunk 2 inserido com sucesso.
Chunk 3 inserido com sucesso.
Chunk 4 inserido com sucesso.
Chunk 5 inserido com sucesso.
Chunk 6 inserido com sucesso.
Chunk 7 inserido com sucesso.
Chunk 8 inserido com sucesso.
Chunk 9 inserido com sucesso.
Chunk 10 inserido com sucesso.
Chunk 11 inserido com sucesso.
Chunk 12 inserido com sucesso.
Chunk 13 inserido com sucesso.
Chunk 14 inserido com sucesso.
Chunk 15 inserido com sucesso.
Chunk 16 inserido com sucesso.
Chunk 17 inserido com sucesso.
Chunk 18 inserido com sucesso.
Chunk 19 inserido com sucesso.
Chunk 20 inserido com sucesso.
Chunk 21 inserido com sucesso.
Chunk 22 inserido com sucesso.
Chunk 23 inserido com sucesso.
Chunk 24 inserido com sucesso.
Chunk 25 inserido com sucesso.
Chunk 26 inserido com sucesso.
Chunk 27 inserido com sucesso.
Chunk 28 inserido com sucesso.
Chunk 29 inserido com sucesso.
Chunk 30 inserido com sucesso.
Chunk 31 inserido com sucesso.
Chunk 32 inserido com sucesso.
Chunk 33 inserido